In [ ]:
# importing the required libraries

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import roc_curve, roc_auc_score
import random
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support

In [ ]:
# Setting the random seeds for the reproducibility of the results
# The segments of the code were taken from
# https://keras.io/api/utils/python_utils/#set_random_seed-function
# https://github.com/keras-team/keras/

tf.random.set_seed(10)
np.random.seed(10)
random.seed(10)

In [ ]:
# Data Preparation
img_size = (128, 128)
batch_size = 64

train_datagen = ImageDataGenerator(rescale=1.0/255) # normalizing images to the range [0, 1] by dividing pixel values by 255.
test_datagen = ImageDataGenerator(rescale=1.0/255)
validate_datagen = ImageDataGenerator(rescale=1.0/255)

# Generating batches of images and labels from the specified directory
train_generator = train_datagen.flow_from_directory(
    'dataset/aug2/train',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = validate_datagen.flow_from_directory(
    'dataset/aug2/val',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    'dataset/aug2/test',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False # For test data, with shuffle=False, is used to ensure predictions align with true labels for evaluation.
)

In [ ]:
# MobileNetV2 Model

# Model Building
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x) # Reduces the dimensions of the feature maps to a single vector by averaging over spatial dimensions (width and height or feature map of image)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False # Freezes the layers of MobileNetV2 to prevent them from being updated during training and will only train the new added top layers.


In [ ]:
# Compiling the model

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Print the model summary
model.summary()

In [ ]:
# Training the model

model_checkpoint = ModelCheckpoint('mobilenetv2_best_model.keras', save_best_only=True, monitor='val_loss')

history = model.fit(
    train_generator,
    epochs=60,
    validation_data=validation_generator,
    callbacks=[model_checkpoint]
)

In [ ]:
# Evaluate the Model

model.load_weights('mobilenetv2_best_model.keras')
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


In [ ]:
# Calculate Precision, Recall, and F1-Score
test_generator.reset() # to ensure that the generator starts from the beginning of the dataset and produces predictions for the entire test set correctly
Y_pred = model.predict(test_generator) # Predicts labels for the test data.
y_pred = (Y_pred > 0.5).astype(int) # Converts predicted probabilities to binary labels (converting to integer array)
y_true = test_generator.classes # providing the true labels for the images in the test set

precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

In [ ]:
# Plot Training & Validation Accuracy

plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training Accuracy', 'Validation Accuracy'], loc='lower right')
plt.xlim(-.1, 60)
plt.ylim(0.82, 1.01)
plt.savefig('model_accuracy.png')
plt.show()

In [ ]:
# Plot Training & Validation Loss

plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training Loss', 'Validation Loss'], loc='upper right')
plt.xlim(0, 60)
plt.ylim(-0.09, 0.8)
plt.savefig('model_loss.png')
plt.show()

In [ ]:
# Confusion Matrix

# Plot Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=test_generator.class_indices.keys())
disp.plot(cmap=plt.cm.Blues)
plt.savefig('confusion_matrix.png')
plt.show()

In [ ]:
# Compute ROC Curve and AUC
fpr, tpr, thresholds = roc_curve(y_true, Y_pred)
auc_score = roc_auc_score(y_true, Y_pred)
print(f"AUC Score: {auc_score:.2f}")

In [ ]:
# Plot ROC Curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {auc_score:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.grid()
plt.savefig('roc_auc_curve.png')
plt.show()

In [ ]:
np.save('y_true_model3.npy', y_true)
np.save('y_pred_proba_model3.npy', y_pred)